In [3]:
import gym
from gym import error, spaces, utils
from gym.utils import seeding

import numpy as np
import matplotlib.pyplot as plt

In [51]:
class Board:
    """
    Describes the board of the game.
    
    Args:
        shape: tuple of ints
    """
    def __init__(self, shape=(12,12)):
        self.shape = shape
        self.board = np.zeros(self.shape, dtype=int) # -1: food, 0: empty field, 1: snake, 2: wall
        
        self._prepare_borders()
        
        self.snakes = []
        
    def _prepare_borders(self):
        self.board[:,0]  = 2
        self.board[:,-1] = 2
        self.board[0,:]  = 2
        self.board[-1,:] = 2
        
    def _reset_snakes(self):
        self.board[self.board == 1] = 0
        
    def _put_snakes(self):
        self._reset_snakes()
        
        for snake in self.snakes:
            for pos in snake.body:
                self.board[pos[0], pos[1]] = 1
                
    def _generate_apple(self):
        # simple implementation
        valid_field = False
        while not valid_field:
            x = np.random.randint(1, self.shape[0] - 1)
            y = np.random.randint(1, self.shape[1] - 1)
            valid_field = self.board[x, y] == 0
        
        self.board[x, y] = 3 # used to be -1
        
    def show(self):
        for row in self.board:
            print(row)

class Snake:
    """
    No step on snek.
    """
    def __init__(self, board, starting_position=None, starting_orientation=3, starting_length=3):
        self.board = board
        self.board.snakes.append(self) # Make the board track the snake
        
        if starting_position is None:
            starting_position = tuple(map(lambda x: x//2, board.shape))
            
        self.position = np.array(starting_position) # 2-tuple of ints
        self.orientation = starting_orientation # 0: down, 1: left, 2: up, 3: right
        
        self.converter = {
            0: np.array([1, 0]),
            1: np.array([0, -1]),
            2: np.array([-1, 0]),
            3: np.array([0, 1]),
        }
        
        self.body = [self.position - 2*self.converter[self.orientation], 
                     self.position - 1*self.converter[self.orientation], 
                     self.position - 0*self.converter[self.orientation]]
        
        self.fed = False
        
    def move(self):
        self.position = self.position + self.converter[self.orientation]
        self.body.append(self.position)
        if self.fed:
            self.fed = False
        else:
            self.body.pop(0)
        
    def __str__(self):
        return "Snake at " + str(self.position)
    
    def __repr__(self):
        return "Snake at " + str(self.position)
        

In [56]:
board = Board()
snake = Snake(board, starting_orientation=3)

board._put_snakes()

board._generate_apple()

board.show()

board.snakes[0].move()
board.snakes[0].orientation = 2
board.snakes[0].move()
print()
board._put_snakes()

board.show()

[2 2 2 2 2 2 2 2 2 2 2 2]
[2 0 0 0 0 0 0 0 0 0 0 2]
[2 0 0 0 0 0 0 0 0 0 3 2]
[2 0 0 0 0 0 0 0 0 0 0 2]
[2 0 0 0 0 0 0 0 0 0 0 2]
[2 0 0 0 0 0 0 0 0 0 0 2]
[2 0 0 0 1 1 1 0 0 0 0 2]
[2 0 0 0 0 0 0 0 0 0 0 2]
[2 0 0 0 0 0 0 0 0 0 0 2]
[2 0 0 0 0 0 0 0 0 0 0 2]
[2 0 0 0 0 0 0 0 0 0 0 2]
[2 2 2 2 2 2 2 2 2 2 2 2]

[2 2 2 2 2 2 2 2 2 2 2 2]
[2 0 0 0 0 0 0 0 0 0 0 2]
[2 0 0 0 0 0 0 0 0 0 3 2]
[2 0 0 0 0 0 0 0 0 0 0 2]
[2 0 0 0 0 0 0 0 0 0 0 2]
[2 0 0 0 0 0 0 1 0 0 0 2]
[2 0 0 0 0 0 1 1 0 0 0 2]
[2 0 0 0 0 0 0 0 0 0 0 2]
[2 0 0 0 0 0 0 0 0 0 0 2]
[2 0 0 0 0 0 0 0 0 0 0 2]
[2 0 0 0 0 0 0 0 0 0 0 2]
[2 2 2 2 2 2 2 2 2 2 2 2]


In [ ]:
class Snake1PEnv(gym.Env):
    """
    Environment for a 1-Player Snake game. 
    """
    metadata = { 'render.modes': ['human'] }

    def __init__(self, board_shape=(12, 12)):
        self.board_shape = board_shape
        self.info = {}
        self._reset()

    def _reset(self):
        self.board = Board(self.board_shape)
        self.snake = Snake(self.board)
        self.reward = 0
        self.done = False
        
        board._put_snakes() 
        board._generate_apple()
        
        return self.board

    def _step(self, action):
        
        self.snake.move()
        
        # if snake hits wall
        if x >= self.board_shape[0] or x < 0 or y >= self.board_shape[1] or y < 0:
            self.done = True

        # if snake hits self
        if self.board[x, y] == 1:
            self.done = True

        # if snake eats fruit
        if self.board[x, y] == 2:
            self.reward += 1
            self._place_fruit()
        else:
            self.snake.pop()

        new_head = [x, y];
        self.snake.insert(0, new_head)
        self._update_snake_position()

        return self.board, self.reward, self.done, self.info

    def _render(self, mode='human', close=False):
        plt.ion()
        plt.imshow(self.board)